# Import Libraries & Files

In [1]:
import pandas as pd
df_raw = pd.read_excel('/workspaces/portfolio/project_clean_postcode/raw_data.xlsx')
df_code = pd.read_excel('/workspaces/portfolio/project_clean_postcode/postcode_clean.xlsx', sheet_name='รหัสไปรษณีย์')

# Fix Wrong Postcode on Raw Data

## Total Raw Data

In [2]:
df_raw.shape[0]

1095

## Fix Raw Data That Not Included in Postcode Data

In [3]:
print("Postcode: " + str(set(df_raw['postcode']).difference(set(df_code['PostCode']))))
print("Province: " + str(set(df_raw['province_name']).difference(set(df_code['ProvinceThai']))))
print("District: " + str(set(df_raw['district_name']).difference(set(df_code['DistrictThaiShort']))))
print("Sub District: " + str(set(df_raw['sub_district_name']).difference(set(df_code['TambonThaiShort']))))

Postcode: set()
Province: set()
District: set()
Sub District: {'หนอง ปรือ'}


In [4]:
df_raw['sub_district_name'] = df_raw['sub_district_name'].str.replace('หนอง ปรือ','หนองปรือ')

## Create Column 'number' to Use as Key Join

In [5]:
df_raw['number'] = range(len(df_raw))

## Merge Data Between Raw Data and Postcode Data

In [6]:
df_merge = df_raw.merge(
    df_code[['PostCode', 'TambonThaiShort', 'DistrictThaiShort', 'ProvinceThai']],
    left_on=['postcode', 'sub_district_name', 'district_name', 'province_name'],
    right_on=['PostCode', 'TambonThaiShort', 'DistrictThaiShort', 'ProvinceThai'],
    how='left'
)

## List Data Which Is Not Match With Postcode Data

In [7]:
list_not_match = df_merge.loc[df_merge['PostCode'].isna(),].index.to_list()

## Merge Data Between Raw Data and Postcode Data Without Postcode

In [8]:
df_merge_drop_postcode = df_raw.loc[df_raw['number'].isin(list_not_match)].merge(
    df_code[['PostCode', 'TambonThaiShort', 'DistrictThaiShort', 'ProvinceThai']],
    left_on=['sub_district_name', 'district_name', 'province_name'],
    right_on=['TambonThaiShort', 'DistrictThaiShort', 'ProvinceThai'],
    how='left'
)

## Replace Postcode from Postcode Data to Raw Data

In [9]:
df_merge.loc[df_merge['number'].isin(list_not_match),'postcode']

6       10150
29      74110
76      17000
114     24130
150     24130
232     21110
293     10510
393     73110
398     10540
551     10200
554     10130
611     10540
694     73140
708     20150
714     10210
811     10250
838     10540
869     18250
908     10540
956     24130
957     72000
981     10200
986     10540
1049    10130
1061    20170
Name: postcode, dtype: int64

In [10]:
df_merge.loc[df_merge['number'].isin(list_not_match),'postcode'] = df_merge_drop_postcode.loc[df_merge_drop_postcode['number'].isin(list_not_match),'PostCode'].to_list()

In [11]:
df_merge.loc[df_merge['number'].isin(list_not_match),'postcode']

6       10130
29      27260
76      17110
114     24180
150     24180
232     21170
293     30150
393     73160
398     10570
551     10230
554     13130
611     10570
694     73180
708     12150
714     10220
811     12150
838     10570
869     18150
908     10570
956     24180
957     72150
981     10220
986     10570
1049    13130
1061    22170
Name: postcode, dtype: int64

In [15]:
df_merge.iloc[:,0:-5]

,province_name,district_name,sub_district_name,postcode
0,สุพรรณบุรี,เมืองสุพรรณบุรี,โคกโคเฒ่า,72000
1,สมุทรปราการ,บางพลี,บางโฉลง,10540
2,ชุมพร,หลังสวน,พ้อแดง,86110
3,สุพรรณบุรี,อู่ทอง,จรเข้สามพัน,72160
4,ยโสธร,เมืองยโสธร,ในเมือง,35000
...,...,...,...,...
1090,ชัยภูมิ,บำเหน็จณรงค์,โคกเริงรมย์,36160
1091,นนทบุรี,ไทรน้อย,ไทรใหญ่,11150
1092,ระยอง,แกลง,ทุ่งควายกิน,21110
1093,ชัยภูมิ,คอนสวรรค์,ช่องสามหมอ,36140
